In [1]:
import os 


In [2]:
os.getcwd()

'/media/puzan/NewVolume/textsummarizer/Text-summarizer/research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'/media/puzan/NewVolume/textsummarizer/Text-summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataTransformConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:Path


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [7]:
# configuration manager
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config=DataTransformConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config
            


        

        

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk


In [10]:
# components
class DataTransformation:
    def __init__(self,config:DataTransformConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,examplebatch):
        input_encodings=self.tokenizer(examplebatch['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(examplebatch['summary'],max_length=128,truncation=True)
        return {
        'input_ids':input_encodings['input_ids'],
        'attention_mask':input_encodings['attention_mask'],
        'labels':target_encodings['input_ids']
        }
    def convert(self):
        dataset_samsum=load_from_disk(self.config.data_path)
        dataset_samsum_pt=dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        


In [12]:
#pipeline
try:
    config=ConfigurationManager()
    data_transformation_config=config.get_data_transformation_config()
    data_transformation=DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e
    

[2024-09-12 17:33:47,639:INFO:common:yaml_file:config/config.yaml loaded successfully
[2024-09-12 17:33:47,644:INFO:common:yaml_file:params.yaml loaded successfully
[2024-09-12 17:33:47,647:INFO:common:created directory at: artifacts
[2024-09-12 17:33:47,649:INFO:common:created directory at: artifacts/data_transformation


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/puzan/anaconda3/envs/textsumm/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 31209.38 examples/s]
